# Demo of IBM Watson Integration
This demo notebook shows how to integrate the IBM Watson News into Celonis by the example of querying any sort of information conerning Acquisitions and IPOs of the vendors. 

You can find more information about IBM Watson Discovery and the python integration here: https://cloud.ibm.com/apidocs/discovery?code=python

<p style="color:red"><b>Disclaimer</b>:
This notebook is outdated and was only intended as a showcase to present the most commonly used features of PyCelonis.
It does not serve as an out-of-the-box use case!</p>

**1) Import IBM Watson and PyCelonis dependencies**

In [3]:
!pip install ibm_watson
import json
import pandas as pd
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from pycelonis import get_celonis
from pycelonis.pql import PQL, PQLColumn, PQLFilter

     |████████████████████████████████| 407 kB 2.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 68 kB 2.5 MB/s eta 0:00:01
  Created wheel for ibm-watson: filename=ibm_watson-5.2.2-py3-none-any.whl size=403389 sha256=885eb1f0ddeff987b4d080abda7e363032982d7fb52836b32238f1e4f2e6e05c
  Stored in directory: /home/sbrand/.cache/pip/wheels/65/68/42/5f3c4d4d244aa2e6c139fc93bfc2d6c4d7a231b4d5472fda08
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.10.0-py3-none-any.whl size=60914 sha256=361f8c431f913d3e20bef37b392497fcf3f0ca44d0bf9f6f75d4a9018808c164
  Stored in directory: /home/sbrand/.cache/pip/wheels/25/f3/b1/1da82e99b552ed44485071ba233652b9f46c06a55853eac780
Successfully built ibm-watson ibm-cloud-sdk-core
You should consider upgrading via the '/home/sbrand/Projects/pycelonis/venv/bin/python -m pip install --upgrade pip' command.


**2) Login to Watson News Collection**

In [4]:
# Connect to IBM Watson
authenticator = IAMAuthenticator(INSERT_TOKEN_HERE)
# Connect to Watson Discovery
discovery = DiscoveryV1(
    version='2018-08-01',
    authenticator=authenticator)
# Example of url: 'https://api.eu-gb.discovery.watson.cloud.ibm.com/instances/df20c0c6-762a-45f9-9778-845873dbf923'
discovery.set_service_url(INSERT_DISCOVERY_INSTANCE_URL_HERE)

**3) Connect to System Environment**

In [5]:
environments = discovery.list_environments().get_result()
environments

{'environments': [{'environment_id': 'system',
   'name': 'Watson System Environment',
   'description': 'Shared system data sources',
   'read_only': True}]}

In [6]:
news_environment_id = 'system'

**4) Print list of available configuration**

In [7]:
configurations = discovery.list_configurations(
    environment_id=news_environment_id).get_result()
print(json.dumps(configurations, indent=2))

{
  "configurations": []
}


In [12]:
collections = discovery.list_collections(news_environment_id).get_result()
news_collections = [x for x in collections['collections']]

collection_id = "news-en"
news_en = [c for c in news_collections if c["collection_id"]==collection_id]
news_en

[{'collection_id': 'news-en',
  'name': 'news-en',
  'language': 'en',
  'status': 'active',
  'description': 'Watson News pre-enriched collection of curated news sources v2 (English)'}]

**5) Available news collections**

In [13]:
news_collections

[{'collection_id': 'news-es',
  'name': 'news-es',
  'language': 'es',
  'status': 'active',
  'description': 'Watson News pre-enriched collection of curated news sources v2 (Spanish)'},
 {'collection_id': 'news-ja',
  'name': 'news-ja',
  'language': 'ja',
  'status': 'active',
  'description': 'Watson News pre-enriched collection of curated news sources v2 (Japanese)'},
 {'collection_id': 'news-ko',
  'name': 'news-ko',
  'language': 'ko',
  'status': 'active',
  'description': 'Watson News pre-enriched collection of curated news sources v2 (Korean)'},
 {'collection_id': 'news-fr',
  'name': 'news-fr',
  'language': 'fr',
  'status': 'active',
  'description': 'Watson News pre-enriched collection of curated news sources v2 (French)'},
 {'collection_id': 'news-de',
  'name': 'news-de',
  'language': 'de',
  'status': 'active',
  'description': 'Watson News pre-enriched collection of curated news sources v2 (German)'},
 {'collection_id': 'news-en',
  'name': 'news-en',
  'language': 'e

**6) Connect to Celonis and Datamodel**

In [14]:
celonis = get_celonis()
dm = celonis.get_datamodel(INSERT_DATAMODEL_ID_HERE)

2021-07-07 16:22:54 - pycelonis: Login successful! Hello SimonBrand
2021-07-07 16:22:54 - pycelonis: Your PyCelonis Version VERSION is outdated (Newest Version: 1.5.6). Please upgrade the package via: pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis --upgrade


**7) Query Vendors from Datamodel**

In [15]:
q = PQL()
q += PQLColumn("LFA1.VENDOR_ID","VENDOR_ID")
q += PQLColumn("LFA1.VENDOR_NAME","VENDOR_NAME")
vendor_names = dm.get_data_frame(q)

In [16]:
vendor_names

,VENDOR_ID,VENDOR_NAME
0,3,Celonis
1,4,IBM
2,1,Samsung
3,2,Google


**8) Query watson with default queries for each of the vendor names**

In [19]:
def find_acquisition_and_ipos(vendor_name, vendor_id):
    """ Function that finds all news about acquisitions or IPOs for all the selected vendors"""
    results_cleaned = []
    acquisition_query = f'enriched_text.entities.text:"{vendor_name}"'
    acquisition_filter = "enriched_title.semantic_roles:(action.normalized:acquire,object.entities:(type::Company))"
    ipo_query =f'enriched_text.keywords.text:"IPO"'
    ipo_filter = f'enriched_text.entities.text:"{vendor_name}"'
    query_results = discovery.query(
        news_environment_id,
        collection_id,
        query= acquisition_query,
        filter=acquisition_filter,
        return_fields='extracted_metadata.sha1').get_result()
    for row in query_results["results"]:
        s = {}
        s["title"] = row["title"]
        s["url"] = row["url"]
        s["host"] = row["host"]
        s["sentiment"] = row["enriched_text"]["sentiment"]["document"]["label"]
        #s["author"] = row["author"]
        s["query"] = acquisition_query
        s["filter"] = acquisition_filter
        s["VENDOR_NAME"] = vendor_name
        s["VENDOR_ID"] = vendor_id
        results_cleaned.append(s)
    query_results = discovery.query(
        news_environment_id,
        collection_id,
        query= ipo_query,
        filter=ipo_filter,
        return_fields='extracted_metadata.sha1').get_result()
    for row in query_results["results"]:
        s = {}
        s["title"] = row["title"]
        s["url"] = row["url"]
        s["host"] = row["host"]
        s["sentiment"] = row["enriched_text"]["sentiment"]["document"]["label"]
        #s["author"] = row["author"]
        s["query"] = ipo_query
        s["filter"] = ipo_filter
        s["VENDOR_NAME"] = vendor_name
        s["VENDOR_ID"] = vendor_id
        results_cleaned.append(s)
    return pd.DataFrame(results_cleaned)

In [27]:
df_acquisitions = pd.DataFrame()
for company in vendor_names.iterrows():
    df = find_acquisition_and_ipos(company[1]["VENDOR_NAME"],
                                               company[1]["VENDOR_ID"])
    df_acquisitions = df_acquisitions.append(df)

In [28]:
df_acquisitions.sample(10)

,title,url,host,sentiment,query,filter,VENDOR_NAME,VENDOR_ID
5,IBM to acquire enterprise software firm Turbon...,https://www.thestatesman.com/business/ibm-acqu...,thestatesman.com,positive,"enriched_text.entities.text:""IBM""",enriched_title.semantic_roles:(action.normaliz...,IBM,4
18,LG Energy moves ahead on South Korea IPO that ...,https://auto.economictimes.indiatimes.com/news...,auto.economictimes.indiatimes.com,positive,"enriched_text.keywords.text:""IPO""","enriched_text.entities.text:""Samsung""",Samsung,1
19,Krafton to launch $5 billion plus IPO subseque...,https://www.gadgetsnow.com/tech-news/krafton-t...,gadgetsnow.com,positive,"enriched_text.keywords.text:""IPO""","enriched_text.entities.text:""Samsung""",Samsung,1
12,LG Energy Solution seeks nod for IPO that IFR ...,https://www.marketscreener.com/quote/stock/SK-...,marketscreener.com,positive,"enriched_text.keywords.text:""IPO""","enriched_text.entities.text:""Samsung""",Samsung,1
5,PhonePe in talks to acquire Indian app store I...,http://bestgamingpro.com/phonepe-in-talks-to-a...,bestgamingpro.com,positive,"enriched_text.entities.text:""Samsung""",enriched_title.semantic_roles:(action.normaliz...,Samsung,1
11,how to do an ipo chart,https://zarma.cvpyouththeatre.org/how-to-do-an...,zarma.cvpyouththeatre.org,positive,"enriched_text.keywords.text:""IPO""","enriched_text.entities.text:""Google""",Google,2
7,"Spotify Acquires Podz, a Podcast Discovery Pla...",https://gadgets.ndtv.com/apps/news/spotify-pod...,gadgets.ndtv.com,positive,"enriched_text.entities.text:""Google""",enriched_title.semantic_roles:(action.normaliz...,Google,2
2,Altman Solon acquires Australian TMT strategy ...,https://www.platformcomms.com/altman-solon-acq...,platformcomms.com,positive,"enriched_text.entities.text:""Google""",enriched_title.semantic_roles:(action.normaliz...,Google,2
14,PLTR Stock: Is It A Buy Right Now? Here’s What...,https://bitcoinethereumnews.com/economy/pltr-s...,bitcoinethereumnews.com,positive,"enriched_text.keywords.text:""IPO""","enriched_text.entities.text:""IBM""",IBM,4
4,IBM acquires cloud managed services provider N...,https://www.expresscomputer.in/cloud/ibm-acqui...,expresscomputer.in,positive,"enriched_text.entities.text:""IBM""",enriched_title.semantic_roles:(action.normaliz...,IBM,4


**10) Push Information back to Datamodel and link it to the Vendors table.**

In [29]:
dm.pool.create_table(df_acquisitions, "WATSON_INSIGHTS",
                     column_config=[{"columnName":"url","fieldLength":200,"columnType":"STRING"},
                                   {"columnName":"title","fieldLength":200,"columnType":"STRING"}],
                    if_exists="drop")

2021-07-07 16:27:52 - pycelonis: Data push job started...
2021-07-07 16:27:55 - pycelonis: Data push job status finished: DONE


{'id': '4b496f5f-4f0d-45a1-aef2-cdde0baca4f5',
 'targetName': 'WATSON_INSIGHTS',
 'lastModified': 1625668075147,
 'lastPing': None,
 'status': 'DONE',
 'type': 'REPLACE',
 'fileType': None,
 'targetSchema': '3f75f7f1-35e6-4a36-8621-7b78d210ea35',
 'upsertStrategy': 'UPSERT_WITH_UNCHANGED_METADATA',
 'fallbackVarcharLength': None,
 'dataPoolId': '3f75f7f1-35e6-4a36-8621-7b78d210ea35',
 'connectionId': None,
 'postExecutionQuery': None,
 'sanitizedPostExecutionQuery': None,
 'keys': [],
 'logs': [],
 'csvParsingOptions': None,
 'tableSchema': {'tableName': 'WATSON_INSIGHTS',
  'columns': [{'columnName': 'url',
    'columnType': 'STRING',
    'fieldLength': 200,
    'decimals': 0,
    'pkField': False},
   {'columnName': 'title',
    'columnType': 'STRING',
    'fieldLength': 200,
    'decimals': 0,
    'pkField': False}]},
 'changeDate': 1625668075147,
 'mirrorTargetNames': [],
 'optionalTenantId': None}

In [ ]:
dm.add_table_from_pool("WATSON_INSIGHTS")
dm.create_foreign_key("LFA1", "WATSON_INSIGHTS",[("VENDOR_ID","VENDOR_ID")])
dm.reload(tables=["WATSON_INSIGHTS"])

**Now the table is linked to the vendor instances and can be accessed for each vendor by filtering on it in the Celonis dashboard**
